In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Project/2023/KT_ETRI

/content/drive/MyDrive/Project/2023/KT_ETRI


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
from xgboost import XGBRegressor

In [4]:
pip install optuna

In [5]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [6]:
import warnings
warnings.filterwarnings('ignore')

### 전처리 마친 데이터 불러오기


In [7]:
train_df = pd.read_csv('Q1_data/Q1_train_pre.csv')
test_df = pd.read_csv('Q1_data/Q1_test_pre.csv')
train_df.head()

,datetime,ru_id,scgfailratio,erabaddatt,erabaddratio,endcaddatt,endcaddratio,endcmodbymenbatt,endcmodbymenbratio,endcmodbysgnbatt,...,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
0,2023-05-19 00:00:00,BaseStationA,0.0,9,100.0,5,100.0,2,100.0,7,...,306.0,23.6,6.4,17.0,11.0,2083.0,-92.8,11.2,3,1
1,2023-05-19 00:05:00,BaseStationA,8.3,12,100.0,9,100.0,0,0.0,15,...,9385.0,15.6,2.4,19.0,15.0,1936.0,-92.8,10.7,11,2
2,2023-05-19 00:10:00,BaseStationA,0.0,9,100.0,8,100.0,4,100.0,11,...,19534.0,17.5,9.9,28.0,23.0,1874.0,-92.7,10.7,5,3
3,2023-05-19 00:15:00,BaseStationA,0.0,11,100.0,10,100.0,0,0.0,8,...,2260.0,19.9,5.0,16.0,14.0,1815.0,-92.6,9.4,8,2
4,2023-05-19 00:20:00,BaseStationA,0.0,9,100.0,8,100.0,0,0.0,9,...,446.0,17.1,0.6,14.0,10.0,1784.0,-92.8,9.6,8,2


# 모델링

In [8]:
data = train_df.iloc[:, 2:]
data.describe()

,scgfailratio,erabaddatt,erabaddratio,endcaddatt,endcaddratio,endcmodbymenbatt,endcmodbymenbratio,endcmodbysgnbatt,endcmodbysgnbratio,connestabatt,...,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,uenomax
count,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,...,1.374480e+05,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000,137448.000000
mean,2.325792,22.161079,97.319247,17.025872,96.439811,3.223641,66.551252,15.928016,94.734347,7.347149,...,4.980100e+04,17.410996,8.858492,87.892534,35.523020,3915.484292,-92.025809,10.989813,9.409981,3.718825
std,8.017694,22.515226,16.096032,18.095286,18.506481,5.963709,47.150096,18.553752,20.611533,13.722298,...,1.269952e+05,13.782209,11.145691,83.697974,37.782572,2152.033668,1.501092,1.736028,17.130399,2.781932
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-96.600000,0.000000,0.000000,0.000000
25%,0.000000,8.000000,100.000000,7.000000,100.000000,0.000000,0.000000,5.000000,100.000000,1.000000,...,2.582000e+03,13.500000,4.300000,34.000000,14.000000,1863.000000,-92.600000,10.200000,1.000000,2.000000
50%,0.000000,18.000000,100.000000,14.000000,100.000000,2.000000,100.000000,12.000000,100.000000,4.000000,...,1.862400e+04,15.700000,6.900000,62.000000,26.000000,3800.000000,-92.200000,11.200000,4.000000,3.000000
75%,1.400000,30.000000,100.000000,23.000000,100.000000,4.000000,100.000000,22.000000,100.000000,9.000000,...,5.978700e+04,18.500000,10.000000,119.000000,45.000000,5585.000000,-91.500000,12.000000,13.000000,5.000000
max,250.000000,823.000000,100.000000,725.000000,100.000000,348.000000,100.000000,664.000000,100.000000,419.000000,...,1.098665e+07,400.000000,300.000000,4587.000000,1487.000000,9590.000000,0.000000,15.000000,817.000000,74.000000


In [9]:
X, y = data.iloc[:, :-1], data.iloc[:, [-1]]

# train : valid = 7 : 3 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, shuffle = False, random_state = 30)
print(X_train.shape, X_valid.shape)

(96213, 33) (41235, 33)


In [8]:
X_train.head()

,scgfailratio,erabaddatt,erabaddratio,endcaddatt,endcaddratio,endcmodbymenbatt,endcmodbymenbratio,endcmodbysgnbatt,endcmodbysgnbratio,connestabatt,connestabratio,redirectiontolte_coverageout,redirectiontolte_epsfallback,handoveratt,handoverratio,reestabatt,reestabratio,rlculbyte,rlcdlbyte,totprbulavg,totprbdlavg,dlreceivedriavg,dltransmittedmcsavg,airmaculbyte,airmacdlbyte,bler_ul,bler_dl,rachpreamblea,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb
0,0.0,9,100.0,5,100.0,2,100.0,7,100.0,1,100.0,1,0,0,0.0,0,0.0,48.0,244.0,0.0,0.8,1.4,3.5,203.0,306.0,23.6,6.4,17.0,11.0,2083.0,-92.8,11.2,3
1,8.3,12,100.0,9,100.0,0,0.0,15,100.0,1,100.0,0,0,1,100.0,0,0.0,855.0,9003.0,0.1,0.9,2.8,4.4,1181.0,9385.0,15.6,2.4,19.0,15.0,1936.0,-92.8,10.7,11
2,0.0,9,100.0,8,100.0,4,100.0,11,100.0,10,100.0,0,0,1,100.0,0,0.0,177.0,19059.0,0.0,1.1,1.9,8.9,586.0,19534.0,17.5,9.9,28.0,23.0,1874.0,-92.7,10.7,5
3,0.0,11,100.0,10,100.0,0,0.0,8,100.0,2,100.0,0,0,1,100.0,0,0.0,270.0,1957.0,0.0,0.8,2.5,3.9,434.0,2260.0,19.9,5.0,16.0,14.0,1815.0,-92.6,9.4,8
4,0.0,9,100.0,8,100.0,0,0.0,9,100.0,0,0.0,0,0,0,0.0,0,0.0,144.0,348.0,0.0,0.8,2.8,1.6,226.0,446.0,17.1,0.6,14.0,10.0,1784.0,-92.8,9.6,8


In [10]:
# scaling
ss = StandardScaler()

X_train_scale = ss.fit_transform(X_train)
X_valid_scale = ss.transform(X_valid)

### Optuna 튜닝

##### best였지만 지금은 아님

In [24]:
def objectiveXGB(trial: Trial, X_train, y_train, X_valid, y_valid):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth': trial.suggest_int('max_depth', 6, 14),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'gamma': trial.suggest_discrete_uniform('gamma', 0.01, 0.5, 0.01),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 1
    }

    model = XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, verbose = False,
                          eval_metric = 'mae', eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100)

    ## MAE로 Loss 계산
    score = mean_absolute_error(xgb_model.predict(X_valid), y_valid)

    return score

In [25]:
study = optuna.create_study(direction = 'minimize', sampler = TPESampler())

study.optimize(lambda trial : objectiveXGB(trial, X_train_scale, y_train, X_valid_scale, y_valid), n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2023-08-20 07:49:17,419] A new study created in memory with name: no-name-0ed279c1-3a52-429a-b266-a72f4bb14112
[I 2023-08-20 07:49:50,450] Trial 0 finished with value: 0.5027508695768532 and parameters: {'n_estimators': 1409, 'max_depth': 12, 'min_child_weight': 29, 'gamma': 0.44, 'learning_rate': 0.0026788359678218917, 'colsample_bytree': 1.0, 'subsample': 0.6}. Best is trial 0 with value: 0.5027508695768532.
[I 2023-08-20 07:49:53,166] Trial 1 finished with value: 0.5071594250726891 and parameters: {'n_estimators': 856, 'max_depth': 9, 'min_child_weight': 13, 'gamma': 0.4, 'learning_rate': 0.01778936007256281, 'colsample_bytree': 0.6, 'subsample': 0.6}. Best is trial 0 with value: 0.5027508695768532.
[I 2023-08-20 07:49:58,125] Trial 2 finished with value: 0.5061695230156057 and parameters: {'n_estimators': 1956, 'max_depth': 11, 'min_child_weight': 14, 'gamma': 0.18000000000000002, 'learning_rate': 0.019206077546730278, 'colsample_bytree': 0.6, 'subsample': 0.7}. Best is trial 0 

Best trial: score 0.49964718021912274,
params {'n_estimators': 3227, 'max_depth': 14, 'min_child_weight': 5, 'gamma': 0.45, 'learning_rate': 0.0016391609287669414, 'colsample_bytree': 1.0, 'subsample': 0.6}


In [28]:
xgboost1 = XGBRegressor(random_state = 1,
                        **study.best_trial.params)

In [30]:
xgboost1.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 100)

[0]	validation_0-mae:1.81524
[100]	validation_0-mae:1.53233
[200]	validation_0-mae:1.29454
[300]	validation_0-mae:1.10241
[400]	validation_0-mae:0.94944
[500]	validation_0-mae:0.82778
[600]	validation_0-mae:0.73335
[700]	validation_0-mae:0.66202
[800]	validation_0-mae:0.60927
[900]	validation_0-mae:0.57129
[1000]	validation_0-mae:0.54496
[1100]	validation_0-mae:0.52707
[1200]	validation_0-mae:0.51513
[1300]	validation_0-mae:0.50770
[1400]	validation_0-mae:0.50329
[1500]	validation_0-mae:0.50092
[1600]	validation_0-mae:0.49994
[1700]	validation_0-mae:0.49982
[1773]	validation_0-mae:0.50010


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.45, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0016391609287669414,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=14, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=3227, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [31]:
y_pred = xgboost1.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.4997853911165033


##### trials 더 많이

best score

In [66]:
def objectiveXGB(trial: Trial, X_train, y_train, X_valid, y_valid):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth': trial.suggest_int('max_depth', 4, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'gamma': trial.suggest_discrete_uniform('gamma', 0.01, 0.5, 0.01),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 1, 0.1),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'random_state': 1
    }

    model = XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, verbose = False,
                          eval_metric = 'mae', eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100)

    ## MAE로 Loss 계산
    score = mean_absolute_error(xgb_model.predict(X_valid), y_valid)

    return score

In [67]:
study = optuna.create_study(direction = 'minimize', sampler = TPESampler())

study.optimize(lambda trial : objectiveXGB(trial, X_train_scale, y_train, X_valid_scale, y_valid), n_trials = 100)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2023-08-20 10:40:20,491] A new study created in memory with name: no-name-145c3619-2f18-4bde-a093-60facd899f9d
[I 2023-08-20 10:40:21,527] Trial 0 finished with value: 0.5300697474005844 and parameters: {'n_estimators': 2883, 'max_depth': 5, 'min_child_weight': 13, 'gamma': 0.49, 'learning_rate': 0.053620043783782566, 'colsample_bytree': 0.9, 'subsample': 0.7, 'lambda': 0.26506685503174954, 'alpha': 0.006442221371634504}. Best is trial 0 with value: 0.5300697474005844.
[I 2023-08-20 10:40:22,376] Trial 1 finished with value: 0.5382744770129839 and parameters: {'n_estimators': 1284, 'max_depth': 4, 'min_child_weight': 41, 'gamma': 0.39, 'learning_rate': 0.04103464781714239, 'colsample_bytree': 0.9, 'subsample': 0.6, 'lambda': 0.010833779902483664, 'alpha': 0.42094087481233783}. Best is trial 0 with value: 0.5300697474005844.
[I 2023-08-20 10:40:41,323] Trial 2 finished with value: 0.503396493977773 and parameters: {'n_estimators': 727, 'max_depth': 12, 'min_child_weight': 18, 'gamma'

Best trial: score 0.49540137126906303,
params {'n_estimators': 1492, 'max_depth': 15, 'min_child_weight': 2, 'gamma': 0.4, 'learning_rate': 0.0038565758063650217, 'colsample_bytree': 0.7, 'subsample': 0.8, 'lambda': 4.076587859420142, 'alpha': 0.5270409349856588}


In [68]:
xgboost1 = XGBRegressor(random_state = 1,
                        **study.best_trial.params)

In [16]:
xgboost1 = XGBRegressor(random_state = 1,
                        n_estimators = 1492, max_depth = 15, min_child_weight = 2, gamma = 0.4,
                        learning_rate = 0.0038565758063650217, colsample_bytree = 0.7, subsample = 0.8,
                        reg_lambda = 4.076587859420142, alpha = 0.5270409349856588)

In [17]:
xgboost1.fit(X_train_scale, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale, y_valid)], early_stopping_rounds = 100,
             verbose = 100)

[0]	validation_0-mae:1.81090
[100]	validation_0-mae:1.20769
[200]	validation_0-mae:0.84835
[300]	validation_0-mae:0.65038
[400]	validation_0-mae:0.55605
[500]	validation_0-mae:0.51569
[600]	validation_0-mae:0.50060
[700]	validation_0-mae:0.49614
[800]	validation_0-mae:0.49609
[846]	validation_0-mae:0.49658


XGBRegressor(alpha=0.5270409349856588, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.4, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0038565758063650217,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=1492, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [18]:
y_pred = xgboost1.predict(X_valid_scale)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.49572385884563935


In [19]:
def predict_df(model) :
  label_sample = pd.read_csv('Q1_data/Q1_label_sample.csv')

  StationB = test_df[test_df['ru_id'] == 'BaseStationB'].iloc[:, 2:]
  StationJ = test_df[test_df['ru_id'] == 'BaseStationJ'].iloc[:, 2:]

  B_scale = ss.transform(StationB)
  B_pred = model.predict(B_scale)

  J_scale = ss.transform(StationJ)
  J_pred = model.predict(J_scale)

  label_sample['BaseStationB'] = B_pred
  label_sample['BaseStationJ'] = J_pred

  return label_sample

In [20]:
label_sample = predict_df(xgboost1)

In [21]:
label_sample.to_csv('Q1_data/xgb_optuna2.csv', index = False)

### 변수 drop 버전

##### 1.

In [42]:
drop_list = ['reestabatt', 'handoverratio', 'redirectiontolte_epsfallback', 'redirectiontolte_coverageout', 'endcaddratio', 'reestabratio',
             'connestabratio', 'handoveratt', 'endcmodbysgnbratio', 'endcaddatt', 'rssipathavg',
             'airmacdlbyte',
             'endcmodbysgnbatt',
             'scgfailratio',
             'connestabatt']

In [43]:
X_train_drop = X_train.drop(drop_list, axis = 1)
X_valid_drop = X_valid.drop(drop_list, axis = 1)

In [44]:
# scaling
ss = StandardScaler()

X_train_scale2 = ss.fit_transform(X_train_drop)
X_valid_scale2 = ss.transform(X_valid_drop)

In [ ]:
study2 = optuna.create_study(direction = 'minimize', sampler = TPESampler())

study2.optimize(lambda trial : objectiveXGB(trial, X_train_scale2, y_train, X_valid_scale2, y_valid), n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study2.best_trial.value,study2.best_trial.params))

In [47]:
xgboost3 = XGBRegressor(random_state = 1,
                        **study2.best_trial.params)

In [48]:
xgboost3.fit(X_train_scale2, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale2, y_valid)], early_stopping_rounds = 100,
             verbose = 100)

[0]	validation_0-mae:1.80638
[100]	validation_0-mae:0.94807
[200]	validation_0-mae:0.61215
[300]	validation_0-mae:0.52002
[400]	validation_0-mae:0.50591
[500]	validation_0-mae:0.50876
[512]	validation_0-mae:0.50934


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.23, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.006330913532478601,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             n_estimators=972, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [49]:
y_pred = xgboost3.predict(X_valid_scale2)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5057961375855621


##### 2.

In [56]:
drop_list = ['reestabatt', 'handoverratio', 'redirectiontolte_epsfallback', 'redirectiontolte_coverageout', 'endcaddratio', 'reestabratio',
             'connestabratio', 'handoveratt', 'endcmodbysgnbratio', 'endcaddatt', 'rssipathavg',
             'airmacdlbyte',
             'endcmodbysgnbatt']

In [57]:
X_train_drop = X_train.drop(drop_list, axis = 1)
X_valid_drop = X_valid.drop(drop_list, axis = 1)

In [58]:
# scaling
ss = StandardScaler()

X_train_scale2 = ss.fit_transform(X_train_drop)
X_valid_scale2 = ss.transform(X_valid_drop)

In [59]:
study2 = optuna.create_study(direction = 'minimize', sampler = TPESampler())

study2.optimize(lambda trial : objectiveXGB(trial, X_train_scale2, y_train, X_valid_scale2, y_valid), n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study2.best_trial.value,study2.best_trial.params))

[I 2023-08-20 10:26:32,980] A new study created in memory with name: no-name-22aaefdb-0947-4946-8265-d8a3d34f7854
[I 2023-08-20 10:26:35,023] Trial 0 finished with value: 0.5157853261397017 and parameters: {'n_estimators': 3424, 'max_depth': 12, 'min_child_weight': 42, 'gamma': 0.22, 'learning_rate': 0.07655750187369179, 'colsample_bytree': 0.6, 'subsample': 1.0}. Best is trial 0 with value: 0.5157853261397017.
[I 2023-08-20 10:27:03,315] Trial 1 finished with value: 0.5057618670670556 and parameters: {'n_estimators': 1274, 'max_depth': 14, 'min_child_weight': 33, 'gamma': 0.18000000000000002, 'learning_rate': 0.003929437949923126, 'colsample_bytree': 0.9, 'subsample': 0.8}. Best is trial 1 with value: 0.5057618670670556.
[I 2023-08-20 10:27:11,972] Trial 2 finished with value: 0.5034636542813568 and parameters: {'n_estimators': 1789, 'max_depth': 10, 'min_child_weight': 5, 'gamma': 0.41000000000000003, 'learning_rate': 0.005432834247646325, 'colsample_bytree': 1.0, 'subsample': 0.6}. 

Best trial: score 0.5019502420331169,
params {'n_estimators': 2468, 'max_depth': 12, 'min_child_weight': 5, 'gamma': 0.24000000000000002, 'learning_rate': 0.004175164842852249, 'colsample_bytree': 0.8, 'subsample': 0.6}


In [60]:
xgboost4 = XGBRegressor(random_state = 1,
                        **study2.best_trial.params)

In [61]:
xgboost4.fit(X_train_scale2, y_train,
             eval_metric = 'mae', eval_set = [(X_valid_scale2, y_valid)], early_stopping_rounds = 100,
             verbose = 100)

[0]	validation_0-mae:1.81045
[100]	validation_0-mae:1.16793
[200]	validation_0-mae:0.79967
[300]	validation_0-mae:0.61420
[400]	validation_0-mae:0.53570
[500]	validation_0-mae:0.50885
[600]	validation_0-mae:0.50282
[700]	validation_0-mae:0.50437
[703]	validation_0-mae:0.50447


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.24000000000000002, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.004175164842852249, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=2468, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [62]:
y_pred = xgboost4.predict(X_valid_scale2)
valid_mae = mean_absolute_error(y_valid, y_pred)
print('valid set에서의 mae :', valid_mae)

valid set에서의 mae : 0.5027504053061241


In [63]:
def predict_df(model) :
  label_sample = pd.read_csv('Q1_data/Q1_label_sample.csv')

  StationB = test_df[test_df['ru_id'] == 'BaseStationB'].iloc[:, 2:]
  StationJ = test_df[test_df['ru_id'] == 'BaseStationJ'].iloc[:, 2:]

  B_scale = ss.transform(StationB.drop(drop_list, axis = 1))
  B_pred = model.predict(B_scale)

  J_scale = ss.transform(StationJ.drop(drop_list, axis = 1))
  J_pred = model.predict(J_scale)

  label_sample['BaseStationB'] = B_pred
  label_sample['BaseStationJ'] = J_pred

  return label_sample

In [64]:
label_sample = predict_df(xgboost4)

In [65]:
label_sample.to_csv('Q1_data/xgb_drop_optuna.csv', index = False)